# Treinando o modelo na Versão 05

In [1]:
import gymnasium as gym
print(f"Using gymnasium version {gym.__version__}")


Using gymnasium version 0.29.1


In [2]:
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback, CheckpointCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
import numpy as np
import os

print(f"Using stable-baselines3 version {stable_baselines3.__version__}")

Using stable-baselines3 version 2.2.1


In [3]:
from Enviroment.Settings import *
from Enviroment.Manager import Enviroment

In [4]:
import torch as th

print(f"Using torch version {th.__version__}")

Using torch version 2.1.2+cpu


In [5]:
class TensorboardCallback(BaseCallback):
    """
    Custom callback for plotting additional values in tensorboard.
    """

    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.pb_history = []

    def _on_step(self) -> bool:

        # Acessa o ambiente
        env = self.training_env.envs[0].env

        info = env.get_custom_info()

        if info["total_requests"] > 39998:
            self.logger.record("custom_on_episode/total_requests", info["total_requests"])
            self.logger.record("custom_on_episode/blocking_probability", info["blocking_probability"])
            self.pb_history.append(info["blocking_probability"])
            self.logger.record("custom_on_episode/blocking_probability_total_mean", np.mean(self.pb_history))

            # Salva a média da probabilidade de bloqueio dos últimos 10 episódios
            self.logger.record("custom_on_episode/blocking_probability_mean_10", np.mean(self.pb_history[-10:]))


            self.logger.record("custom_on_episode/reward_episode", info["reward_episode"])
            self.logger.record("custom_on_episode/total_number_of_blocks", info["total_number_of_blocks"])
            self.logger.record("custom_on_episode/calls_SAR", env.SAR_calls)

        # Verifica se o episódio terminou	
        return True

In [6]:
# Configura como deve ser o ambiente
enviroment_type = {
    "Observation": "ODD-one-hot",
    "Action": "RSA-SAR",
    "Reward": "RL-defaut",
    "StopCond": "40kReqs",
    "StartCond": "Empty"
}

# Cria o ambiente de simulação
env = Enviroment(
    network_load=300,
    k_routes=K_ROUTES,
    number_of_slots=NUMBER_OF_SLOTS,
    enviroment_type=enviroment_type,
    data_folder="Train_PPO_40kReqs_300_v5",
)

# Cria o log path
LOG_PATH = env.folder_name

print(f"Using the enviroment {enviroment_type}")

print(f"Using the log path {LOG_PATH}")

# Criando o diretório para o monitoramento
monitor_dir = LOG_PATH + "\\monitor_logs"
os.makedirs(monitor_dir, exist_ok=True)

# Criando o monitor para monitorar o ambiente
env = Monitor(env, monitor_dir)

# Cria o dicionário com as configurações da política da rede. 
policy_kwargs = dict(
    activation_fn=th.nn.ReLU,
    net_arch=dict(pi=[128], vf=[128]))

# Create the agent
model = PPO(
    "MlpPolicy", 
    env,
    policy_kwargs=policy_kwargs,
    learning_rate=3.240817843698623e-05,
    gamma=0.9991972343214264,
    ent_coef=0.0001899484641387907,
    clip_range=0.13399956493573578,
    vf_coef=0.6395307341097642,
    max_grad_norm=1.1240988009356605,
    gae_lambda=0.8993673780839393,
    n_epochs=6,
    clip_range_vf=0.2327530358366276,
    batch_size=256,
    n_steps=128,
    verbose=0, 
    tensorboard_log=LOG_PATH + '\\tensorboard\\', 
    seed=42
)

# Callback para salvar o melhor modelo
best_model_callback = CheckpointCallback(
    save_freq=40_000, 
    save_path=LOG_PATH + '/training/models/', 
    name_prefix='models',
    verbose=2
)

# Callback de avaliação
eval_callback = EvalCallback(
    env, 
    best_model_callback, 
    eval_freq=40_000, 
    deterministic=True, 
    best_model_save_path=LOG_PATH + '\\training\\best_model',
    log_path=LOG_PATH + '\\training\\logs',
    render=False, 
    verbose=2
)

model.learn(
    total_timesteps=5_000_000, 
    callback=[TensorboardCallback(), best_model_callback, eval_callback],
    progress_bar=True, 
    tb_log_name="Model_v5"
)

# Salva o modelo treinado
model.save(LOG_PATH + '\\training\\final_model')


Using the enviroment {'Observation': 'ODD-one-hot', 'Action': 'RSA-SAR', 'Reward': 'RL-defaut', 'StopCond': '40kReqs', 'StartCond': 'Empty'}
Using the log path ../logs/Train_PPO_40kReqs_300_v5_001


d:\98_phD_Files\Projeto 006 - Artigo rede regular iTwo\venv\Lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 256, but because the `RolloutBuffer` is of size `n_steps * n_envs = 128`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 128
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=128 and n_envs=1)
  warnings.warn(


Output()

Saving model checkpoint to ../logs/Train_PPO_40kReqs_300_v5_001/training/models/models_40000_steps.zip

Eval num_timesteps=40000, episode_reward=38967.60 +/- 37.19

...

Eval num_timesteps=4960000, episode_reward=39040.40 +/- 88.72

Episode length: 40000.00 +/- 0.00

Saving model checkpoint to ../logs/Train_PPO_40kReqs_300_v5_001/training/models/models_4999996_steps.zip

Eval num_timesteps=5000000, episode_reward=39028.40 +/- 48.57

Episode length: 40000.00 +/- 0.00